# Novartis - List
## Beachten
* PDF zuerst entschlüsseln!
* Für die OCR-Erkennung wurde ABBYY Fine Reader for Mac verwendet (Lizenz vorhanden)
* Wichtig: Sprachen einstellen auf: Deutsch, Englisch, Italienisch, Französisch
* Als CSV exportieren
* HCP & HCO **manuell** setzen

In [2]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib
import re

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *
import consts

## Unlock PDF

In [1]:
import pikepdf
pdf = pikepdf.open('payment-disclosure-pharma-switzerland-2018-de.pdf')
pdf.save('unlocked.pdf')

## Import CSV

In [16]:
# Read CSV into DataFrame
header = ['empty', 'name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'empty1', 'total']  
df = pd.read_csv("payment-disclosure-pharma-switzerland-2018-de.csv", sep=";", header=None, names=header)

## Format Table

In [20]:
df_export = df.copy()

#clear first cell, if total notna
df_export.loc[df_export['total'].notna(), 'empty'] = np.NaN

#Shift
df_export[df_export['empty'].notna()] = df_export[df_export['empty'].notna()].shift(1, axis='columns')

#Remove empty
df_export.drop(columns=['empty', 'empty1'], inplace=True)

#Replace str "TOTAL" to nan
df_export.loc[df_export['total'].str.contains('TOTAL', na=False), 'total'] = np.nan

#Remove rows which have no values
df_export = df_export.dropna(subset=['total'], how='all')
df_export = df_export.reset_index(drop=True)

#Remove \r
df_export = remove_carination(df_export, " ")

#Set Type
index_hco = df_export[df_export['name'].str.contains("aarReha", na=False)].index[0]
df_export['type'] = np.where(df_export.index < index_hco, 'hcp', 'hco')

#Revert name
df_export = revert_name(df_export, ' ')

#Clean Numbers
df_export = cleanup_number(df_export)
df_export = remove_dots(df_export)
df_export = remove_in_numbers(df_export, '-')
df_export = replace_comma_to_dot(df_export)

#basic string conversion
df_export = basic_string_conversion(df_export)

#Add PLZ
df_export = add_plz(df_export)


In [21]:
def fix_errors(df_export, field):
    other_fields = number_fields.copy()
    other_fields.remove(field)
    
    #Loop each row
    for index, row in df_export[df_export[field] == 'ERROR'].iterrows():
        
        #Check if other fields are numeric. If not, break
        has_strings = False
        for f in other_fields:
            if re.match(r'^[0-9.]*$', str(row[f])) == None:
                has_strings = True
                break
            
        #if no strings, calculate!
        if not has_strings:
            if field != 'total':
                without_total = other_fields.copy()
                without_total.remove('total')
                df_export.loc[index, field] = (
                    pd.to_numeric(df_export.loc[index, 'total']) 
                    - pd.to_numeric(df_export.loc[index, without_total[0]])
                    - pd.to_numeric(df_export.loc[index, without_total[1]])
                    - pd.to_numeric(df_export.loc[index, without_total[2]])
                    - pd.to_numeric(df_export.loc[index, without_total[3]])
                    - pd.to_numeric(df_export.loc[index, without_total[4]])
                )
            else:
                without_total = other_fields.copy()
                df_export.loc[index, field] = (
                    pd.to_numeric(df_export.loc[index, without_total[0]])
                    + pd.to_numeric(df_export.loc[index, without_total[1]])
                    + pd.to_numeric(df_export.loc[index, without_total[2]])
                    + pd.to_numeric(df_export.loc[index, without_total[3]])
                    + pd.to_numeric(df_export.loc[index, without_total[4]])
                    + pd.to_numeric(df_export.loc[index, without_total[5]])
                )
                           
    return df_export

  
#Fill ERROR
for field in number_fields:
    df_export.loc[df_export[field].str.contains(r'^[0-9.]*$', regex=True) == False, field] = 'ERROR'
   
#Fill na in numeric fields
for f in number_fields:
    df_export[f] = df_export[f].fillna(value=0)

#Fix Errors
for f in number_fields:
    df_export = fix_errors(df_export, f)  

In [24]:
#Export
add_warning(manually=True)
df_export = amounts_to_number(df_export)
export_list(df_export, 'novartis')

Total nicht Summe der Werte
saved


In [26]:
#write_to_csv(df_export, 'tmp.csv', True)
#write_to_excel(df_export, 'tmp.xlsx', open=True)